# Setup môi trường
1. Truy cập [Google AI Studio](https://aistudio.google.com/apikey) và chọn `Create API Key`
2. Tạo file `.env` và lưu API key dưới dạng `GOOGLE_API_KEY="YOUR_API_KEY"`
3. Sử dụng thư viện `python-dotenv` để quản lý API Key

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai

/Users/hieu1966hn/Desktop/MindX/MindX-CS/HDT/HDT-CSI14/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hieu1966hn/Desktop/MindX/MindX-CS/HDT/HDT-CSI14/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

# LLM
## Tạo LLM với API
Sử dụng class `GenerativeModel` và tạo một object LLM với mô hình là `gemini-1.5-flash`

In [4]:
model = genai.GenerativeModel("gemini-1.5-flash")

## Tương tác với LLM
Thử tương tác với mô hình bằng phương thức `generate_content` của đối tượng `model`

In [7]:
prompt = 'Bạn có thể làm gì?'
response = model.generate_content(prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "T\u00f4i l\u00e0 m\u1ed9t m\u00f4 h\u00ecnh ng\u00f4n ng\u1eef l\u1edbn, \u0111\u01b0\u1ee3c \u0111\u00e0o t\u1ea1o b\u1edfi Google. T\u00f4i c\u00f3 th\u1ec3 l\u00e0m nhi\u1ec1u vi\u1ec7c, bao g\u1ed3m nh\u01b0ng kh\u00f4ng gi\u1edbi h\u1ea1n \u1edf:\n\n* **T\u1ea1o v\u0103n b\u1ea3n:** T\u00f4i c\u00f3 th\u1ec3 vi\u1ebft c\u00e2u chuy\u1ec7n, b\u00e0i th\u01a1, b\u00e0i b\u00e1o, email, m\u00e3 code, k\u1ecbch b\u1ea3n v\u00e0 nhi\u1ec1u lo\u1ea1i v\u0103n b\u1ea3n kh\u00e1c theo y\u00eau c\u1ea7u c\u1ee7a b\u1ea1n.  T\u00f4i c\u00f3 th\u1ec3 vi\u1ebft b\u1eb1ng nhi\u1ec1u phong c\u00e1ch kh\u00e1c nhau, t\u1eeb trang tr\u1ecdng \u0111\u1ebfn th\u00e2n m\u1eadt.\n\n* **T\u00f3m t\u1eaft v\u0103n b\u1ea3n:** T\u00f4i c\u00f3 th\u1ec3 t\u00f3m t\u1eaft c\u00

Kết quả sẽ trả về một đối tượng có cấu trúc, và ta quan sát được câu trả lời của mô hình nằm ở phần `result` -> `candidates` -> `content` -> `parts` -> `text`.

Để truy cập nhanh câu trả lời, sử dụng trực tiếp thuộc tính `text` của đối tượng `response`.

In [8]:
response.text

'Tôi là một mô hình ngôn ngữ lớn, được đào tạo bởi Google. Tôi có thể làm nhiều việc, bao gồm nhưng không giới hạn ở:\n\n* **Tạo văn bản:** Tôi có thể viết câu chuyện, bài thơ, bài báo, email, mã code, kịch bản và nhiều loại văn bản khác theo yêu cầu của bạn.  Tôi có thể viết bằng nhiều phong cách khác nhau, từ trang trọng đến thân mật.\n\n* **Tóm tắt văn bản:** Tôi có thể tóm tắt các văn bản dài thành các đoạn ngắn hơn, giữ lại thông tin quan trọng.\n\n* **Dịch thuật:** Tôi có thể dịch văn bản giữa nhiều ngôn ngữ.\n\n* **Trả lời câu hỏi:** Tôi có thể trả lời các câu hỏi của bạn dựa trên kiến thức của tôi, được thu thập từ một lượng lớn dữ liệu văn bản.\n\n* **Tạo nội dung sáng tạo:** Tôi có thể viết truyện cười, câu đố, hoặc thậm chí cả những câu chuyện giả tưởng.\n\n* **Giúp viết code:** Tôi có thể giúp bạn viết code, debug code, hoặc giải thích các đoạn code.\n\n* **Phân tích văn bản:** Tôi có thể phân tích văn bản để tìm hiểu về chủ đề, giọng điệu, hoặc cảm xúc.\n\n* **Đề xuất ý tư

## Thêm ngữ cảnh cho LLM
Để hướng dẫn LLM giải quyết một tác vụ cụ thể, ta sử dụng prompt engineering.

Bạn là chủ một nhà hàng. Hãy viết hướng dẫn phù hợp để LLM của bạn có thể:
1. quảng cáo về nhà hàng
2. giới thiệu menu cho khách hàng

Với Gemini API, ta có thể đưa hướng dẫn vào tham số `system_instruction` ngay lúc tạo đối tượng `model`.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là MienLuon, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng PhoBo Cuisine.
                              
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng PhoBo Cuisine: là một nhà hàng thành lập bởi doanh nghiệp Việt Nam, ở địa chỉ 22 Jump Streets, New York, USA 
                              2. Giới thiệu menu nhà hàng gồm các món: phở, bún bò, cớm tấm, cơm rang, nem nướng, bánh mỳ sốt vang, bánh mỳ pate, cháo sườn, bún chả, ...
                              
                              Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.
                              '
                              """)

Thử lại với prompt đến từ khách hàng.

In [15]:
prompt = 'Địa chỉ nhà hàng'
response = model.generate_content(prompt)
print(response.text)

Nhà hàng PhoBo Cuisine nằm ở địa chỉ 22 Jump Streets, New York, USA



## Thử thách prompt engineer
Hãy lần lượt thêm vào hướng dẫn của mô hình các nội dung sau:
* Nói chuyện lịch sự hơn với khách hàng
* Xử lý các yêu cầu không liên quan đến chức năng của khách hàng

Theo dõi cách mô hình thay đổi câu trả lời khi đã chỉnh sửa hướng dẫn.

In [16]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là MienLuon, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng PhoBo Cuisine.
                              
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng PhoBo Cuisine: là một nhà hàng thành lập bởi doanh nghiệp Việt Nam, ở địa chỉ 22 Jump Streets, New York, USA 
                              2. Giới thiệu menu nhà hàng gồm các món: phở, bún bò, cớm tấm, cơm rang, nem nướng, bánh mỳ sốt vang, bánh mỳ pate, cháo sườn, bún chả, ...
                              
                              Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.
                              
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khách hàng, vì khách hàng là thượng đế.
                              """)

In [ ]:
prompt = "Nhà hàng mình có những món ăn gì dành cho tôi không"
response = model.generate_content(prompt)
response.text

'Chào anh/chị!  Rất vui được chào đón anh/chị đến với PhoBo Cuisine!  Chúng tôi có thực đơn đa dạng với nhiều món ăn Việt Nam ngon miệng, phù hợp với mọi khẩu vị.  Nhà hàng chúng tôi có phở, bún bò, cơm tấm, cơm rang, nem nướng, bánh mì sốt vang, bánh mì pate, cháo sườn, bún chả và nhiều món khác nữa.  Tất cả các món ăn đều được chế biến tươi ngon, đảm bảo vệ sinh an toàn thực phẩm.  Anh/chị có thể xem chi tiết thực đơn trên website của chúng tôi hoặc hỏi nhân viên phục vụ để được tư vấn thêm nhé!  Chúng tôi rất mong được phục vụ anh/chị!\n'

## Kết nối file dữ liệu với LLM

Đọc file dữ liệu từ `menu.csv` vào DataFrame`menu_df`

In [18]:
menu_df = pd.read_csv("./menu.csv", index_col=[0]) 
menu_df

,name,description,ingredients,notes
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò..."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN


Cập nhật hướng dẫn với cột `name` trong `menu_df` và thử lại với prompt mới.

In [19]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=f"""
                              Bạn tên là MienLuon, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng PhoBo Cuisine.
                              
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng PhoBo Cuisine: là một nhà hàng thành lập bởi doanh nghiệp Việt Nam, ở địa chỉ 22 Jump Streets, New York, USA 
                              2. Giới thiệu menu nhà hàng gồm các món: {', '.join(menu_df['name'].to_list())}
                              Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.
                              
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khách hàng, vì khách hàng là thượng đế.
                              """)

In [20]:
from IPython.display import Markdown

prompt = "Liệt kê các món ăn trong menu"
answer = model.generate_content(prompt)
Markdown(answer.text)

Chào mừng bạn đến với PhoBo Cuisine!  Rất vui được hỗ trợ bạn. Dưới đây là menu của chúng tôi:

* Gỏi Cuốn
* Phở Việt Nam
* Cơm Tấm
* Bún Bò
* Khoai Tây Chiên

Bạn muốn tìm hiểu thêm về món nào không ạ?  Chúng tôi rất hân hạnh được phục vụ bạn!


## [Mở rộng] Hệ thống hỏi - đáp
Sử dụng phương thức embed_content để biến DataFrame thành dạng embeddings để LLM có thể kết nối.

In [ ]:
for m in genai.list_models(): # liệt kê các mô hình hiện có trong thư viện genai
    if 'embedContent' in m.supported_generation_methods: # kiểm tra mô hình có hỗ trợ phương thức embedContent không
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [ ]:
model_name = 'models/text-embedding-004' # chọn mô hình

def embed_column(title, text): # gửi văn text và tiêu đề đến mô hình embedding 
    return genai.embed_content(model=model_name, # => vector số hóa (embedding)
                               content=text,
                               task_type='retrieval_document', # giúp mô hình hiểu: dùng trong hệ thống hỏi - đáp dựa trên tìm kiếm nội dung
                               title=title) ['embedding']
    
menu_df['description_emb'] = menu_df.apply(lambda row: embed_column(row['name'], row['description']), axis=1) # duyệt từng row của DF menu_df

menu_df

,name,description,ingredients,notes,description_emb
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...,"[-0.013258968, 0.072852395, -0.0081224935, -0...."
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.,"[-0.015033179, 0.04973999, -0.0054149763, -0.0..."
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...,"[0.009709443, 0.078435704, -0.027416762, -0.04..."
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò...","[-0.0042047882, 0.059224624, -0.000562346, -0...."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN,"[0.009064724, 0.056260645, -0.030308044, -0.05..."


In [25]:
prompt = "Có món nào có phở không?"
response = model.generate_content(prompt)
response.text

'Chào mừng bạn đến với PhoBo Cuisine!  Vâng ạ, nhà hàng chúng tôi có món Phở Việt Nam rất nổi tiếng đấy ạ.  Ngoài ra, chúng tôi còn phục vụ nhiều món ăn Việt Nam khác nữa, mời bạn xem qua menu để lựa chọn nhé!\n'